In [30]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


c:\Users\Lenovo\OneDrive - Universidad EAFIT\Documentos\Obsidian\Obsidian\2024-1\Cuántica\ps\notas\.venv\Scripts\python.exe: No module named pip


In [31]:
import numpy as np
from numpy import ndarray
from numpy import sqrt, log2,ceil
from typing import Callable

In [32]:
N = 5


def f(i):
    return i==0


In [33]:
def H_otimes_n(n) -> ndarray:
    hadamard: ndarray = np.sqrt(0.5) * np.array([[1, 1], [1, -1]])
    result = hadamard
    n -= 1  # primera iteración siempre inicia en hadamard, pero no cuenta como multiplicación

    for _ in range(n):
        result = np.kron(result, hadamard)
    # Kronecker products crea submatrices, esto reshapea a matriz
    op = result.reshape(-1, result.shape[-1])
    return op

In [34]:
def phase_inversion(N, f:Callable)-> ndarray:
    data = list(-1 if f(x) else 1 for x in range(N))
    U_f = np.diag(data)
    return U_f

In [35]:
def proyection_about_mean(N):
    # Projector |0^n><0^n|
    projector = np.zeros((N, N))
    projector[0, 0] = 1
    # Identity matrix I
    identity = np.eye(N)

    # Grover diffusion operator: U_0^n = 2|0^n><0^n| - I
    reflection_about_mean = 2 * projector - identity
    return reflection_about_mean


In [36]:
def twice(M: ndarray):
    identity = np.eye(2)
    M = np.kron(identity, M)
    #M = M.reshape((identity.shape[0] * M.shape[0], identity.shape[1] * M.shape[1]))
    return M
def attach_auxiliar_vector(state:ndarray):
    aux = np.zeros_like(state)
    aux[1][0] = 1
    return np.vstack((state,aux))

In [37]:
def grover(NumberStates, f: Callable, aux:bool = True):
    n = int(ceil(log2(NumberStates)))
    N = int(np.power(2, n))
    ket_h = np.zeros((N, 1))
    ket_h[0][0] = 1
    H = H_otimes_n(n)
    reflect = phase_inversion(N, f)
    mean_reflection = proyection_about_mean(N)
    if aux:
        H = twice(H)
        reflect = twice(reflect)
        mean_reflection = twice(mean_reflection)
        ket_h = attach_auxiliar_vector(ket_h)
    

    ket_h = H @ ket_h
    for i in range(int(sqrt(N))):
        ket_h = reflect @ ket_h

        ket_h = H @ ket_h

        ket_h = mean_reflection @ ket_h

        ket_h = H @ ket_h
    if aux:
        return np.vsplit(ket_h,2)
    return ket_h


grover(N, f)


[array([[ 0.97227182],
        [-0.08838835],
        [-0.08838835],
        [-0.08838835],
        [-0.08838835],
        [-0.08838835],
        [-0.08838835],
        [-0.08838835]]),
 array([[-0.08838835],
        [-0.44194174],
        [ 0.26516504],
        [-0.44194174],
        [ 0.26516504],
        [-0.44194174],
        [ 0.26516504],
        [-0.44194174]])]

Chequear que el código funciona en todos los casos cuando se busca un valor hasta el valor 16.
Chequear como funciona el código para cuando se buscan 2 valores (todas las permutaciones) hasta el valor 16

In [38]:
# Esta vaina se quiebra con más de dos elementos btw
def result(N, f, aux):
    if aux:
        psi = grover(N, f, aux)[0]
    else:
        psi = grover(N, f, aux)
    bitmask = psi > 0
    # Filter the original array using boolean indexing and get the index as well
    found = [list(psi[bitmask]), np.nonzero(bitmask)[0].tolist()]
    if aux:
        return found[1]  # , psi[1]
    return found[1]


# Chequear para las combinaciones de hasta 2 elementos
def chequear_valores(N_maximo):
    aux = True
    N_maximo += 2
    for N in range(2, N_maximo):
        for i in range(N):
            f = lambda x: x == i
            res = result(N, f, aux)
            assert res == result(N, f, not aux)
            if res:
                print(res)
            else:
                print(f"Nada  se encuentra con {N} estados, buscando el indice {i}")
                raise ValueError(f"No encontró con {N} estados para el valor {i}")
    for N in range(2, N_maximo):
        for i in range(N):
            for j in range(N):
                f = lambda x: x == i or x == j
                res = result(N, f, aux)
                assert res == result(N, f, not aux)
                if res:
                    print(res)
                else:
                    print(
                        f"Nada  se encuentra con {N} estados, buscando indices {i}, {j}"
                    )


chequear_valores(18)  # Chequear permutaciones de hasta 2 números, hasta el 16

[0]
[1]
[0]
Nada  se encuentra con 2 estados, buscando indices 0, 1
Nada  se encuentra con 2 estados, buscando indices 1, 0
[1]
[0]
[1]
[2]
[0]
Nada  se encuentra con 3 estados, buscando indices 0, 1
Nada  se encuentra con 3 estados, buscando indices 0, 2
Nada  se encuentra con 3 estados, buscando indices 1, 0
[1]
Nada  se encuentra con 3 estados, buscando indices 1, 2
Nada  se encuentra con 3 estados, buscando indices 2, 0
Nada  se encuentra con 3 estados, buscando indices 2, 1
[2]
[0]
[1]
[2]
[3]
[0]
Nada  se encuentra con 4 estados, buscando indices 0, 1
Nada  se encuentra con 4 estados, buscando indices 0, 2
Nada  se encuentra con 4 estados, buscando indices 0, 3
Nada  se encuentra con 4 estados, buscando indices 1, 0
[1]
Nada  se encuentra con 4 estados, buscando indices 1, 2
Nada  se encuentra con 4 estados, buscando indices 1, 3
Nada  se encuentra con 4 estados, buscando indices 2, 0
Nada  se encuentra con 4 estados, buscando indices 2, 1
[2]
Nada  se encuentra con 4 estados, bu

Ah y tambien chequear que las matrices de los operadores creados sean matrices unitarias, que el vector inicializado sea unitario

In [39]:
def op_deets(op: ndarray, label: str):
    print()
    print(label)
    print(op.shape)
    print(op)
    print()
    print()

def is_op_unitary(A:ndarray, N: int):
    case1 = A.conj().T @ A
    case2 = A @ A.conj().T
    Identity = np.eye(N)
    assert np.allclose(case1, case2) 
    assert np.allclose(case2, Identity)

def is_v_unit(v:ndarray, m):
    v_2 = (v.T @ v)[0][0]
    tol = 1e-10
    is_approximate = tol > abs(v_2 - 1)
    if m:
        print("v_2 inner product")
        print(v_2)
    if not is_approximate:
        print("hay que dividir por", v_2 )
    assert is_approximate

def test_ops(n, N, f, psi, m:bool = False):
    if m:
        print(f"Qubits n: {n}, Estados totales N : {N}")
    H = H_otimes_n(n)
    if m:
        label = "Hadamard_otimes_n:"
        op_deets(H, label)
    is_op_unitary(H, N)
    # Hadamard además de unitaria, es Hermitian?
    assert np.allclose(H, H.conj().T)
    # P_I es una función oráculo
    P_I = phase_inversion(N, f)
    if m:
        label = "Phase inversion:"
        op_deets(P_I, label)
    is_op_unitary(P_I, N)

    is_v_unit(psi,m)
    if m:
        print("psi es unitario")
        print(psi)

    U_0n = proyection_about_mean(N)
    if m:
        label = "Proyection about mean:"
        op_deets(U_0n, label)
        is_op_unitary(U_0n, N)

n = int(ceil(log2(N)))
N_s = int(np.power(2, n))
H = H_otimes_n(n)
ket_h = np.zeros((N_s, 1))
ket_h[0][0] = 1
ket_h = H @ ket_h
test_ops(n, N_s, f, ket_h)

